# Bayesian network model of predator-prey interactions

## 1. Description

Here, I build a Bayesian network implementation of the DAG where there is no feedback between the predator and the prey's behavior within a match.

Our objective is to use causal inference to model the predator-prey interaction as it changes throughout the predator's experience. Our DAG explains how we expect the system to work.

## 2. Prepare the session

I start by importing the librairies that I'll need to build the network.

In [1]:
# For Bayesian networks
import bnlearn as bn

# For data manipulations
from pathlib import Path
import pandas as pd
import numpy as np

## 3. Import the data

I import the data to build the network.

In [2]:
# Specify the parent folder
notebook_dir = Path.cwd()
parent_folder = notebook_dir.parent.parent

# Data path
data_subfolder = "data"
file_name = "FraserFranco-etal-XXXX.csv"

# Full path to data file
file_path = parent_folder / data_subfolder / file_name

# Import dataset
dat = pd.read_csv(file_path, index_col = None)

### 3.1 Visualize the data structure

Let's start by checking the data

In [3]:
dat.head()

,match_id,predator_id,xp_level,timestamp,game_mode,realm_id,environment_id,avatar_id,game_duration,prey_avg_rank,...,predicted_mean_speed,predicted_sigma_speed,cluster,hunting_type,chase_count,chase_count_success,attacks_count,prey_avg_heal_count,prey_var_heal_count,chase_success_ratio
0,002W3HHO,pred558338,interm,2021-01-14 22:33:06,Online,The Macmillan Estate,Suffocation Pit,Spirit,523.008972,5.50,...,-0.092314,0.283652,1,ambush-flexible,17,8,17,2.250000,2.916667,0.470588
1,005QEKAB,pred390909,novice,2020-12-30 09:14:30,Online,Backwater Swamp,The Pale Rose,Trapper,802.905701,4.50,...,-0.109873,0.286322,1,ambush-flexible,27,7,17,2.666667,4.333333,0.259259
2,00ATEFT5,pred389952,advanced,2021-04-25 11:57:02,Online,The Macmillan Estate,Suffocation Pit,Oni,693.849854,4.25,...,0.032009,-0.138640,1,intercepting-intermediate,30,6,11,3.750000,10.916667,0.200000
3,00BQMP7T,pred35544,advanced,2021-03-17 03:01:23,Online,Léry's Memorial Institute,Treatment Theatre,Executioner,907.008728,8.50,...,0.024512,-0.061111,1,intercepting-intermediate,20,8,11,2.500000,13.666667,0.400000
4,00CQOXUD,pred199640,novice,2020-12-15 10:42:03,Online,Autohaven Wreckers,Blood Lodge,Hillbilly,605.748718,15.00,...,-0.009210,0.140301,3,intercepting-intermediate,24,6,12,3.500000,20.333333,0.250000


In [4]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100062 entries, 0 to 100061
Data columns (total 26 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   match_id               100062 non-null  object 
 1   predator_id            100062 non-null  object 
 2   xp_level               100062 non-null  object 
 3   timestamp              100062 non-null  object 
 4   game_mode              100062 non-null  object 
 5   realm_id               100062 non-null  object 
 6   environment_id         100062 non-null  object 
 7   avatar_id              100062 non-null  object 
 8   game_duration          100062 non-null  float64
 9   prey_avg_rank          100062 non-null  float64
 10  total_xp_pred          100062 non-null  int64  
 11  cumul_xp_pred          100062 non-null  int64  
 12  hunting_success        100062 non-null  int64  
 13  points_predator        100062 non-null  int64  
 14  pred_speed             100062 non-nu

In [5]:
print("Colum names:", dat.columns)

Colum names: Index(['match_id', 'predator_id', 'xp_level', 'timestamp', 'game_mode',
       'realm_id', 'environment_id', 'avatar_id', 'game_duration',
       'prey_avg_rank', 'total_xp_pred', 'cumul_xp_pred', 'hunting_success',
       'points_predator', 'pred_speed', 'prey_avg_speed',
       'predicted_mean_speed', 'predicted_sigma_speed', 'cluster',
       'hunting_type', 'chase_count', 'chase_count_success', 'attacks_count',
       'prey_avg_heal_count', 'prey_var_heal_count', 'chase_success_ratio'],
      dtype='object')


## 4. Build a simple Bayesian network

Here, I use my expertise and knowledge of the system to build the network. In this manuscript, we suggest a causal model of predator-prey interactions, and we thus build our model based on our beliefs on how the system works.

### 4.1 Select variables for the Bayesian network

I start by subsetting the variables that I will use for the model.

In [6]:
# Select relevant variables
selected_variables = [
    "xp_level",
    "hunting_type",
    "hunting_success"
]

# Subset the data
subset = dat[selected_variables]
subset

,xp_level,hunting_type,hunting_success
0,interm,ambush-flexible,4
1,novice,ambush-flexible,1
2,advanced,intercepting-intermediate,0
3,advanced,intercepting-intermediate,4
4,novice,intercepting-intermediate,2
...,...,...,...
100057,interm,intercepting-intermediate,3
100058,advanced,cursorial-specialist,1
100059,interm,intercepting-intermediate,2
100060,interm,cursorial-specialist,2


### 4.2 Build the Bayesian network structure

Here, I build the relationships (edges) between the variables (nodes).

The model goes like this :

experience --> hunting type --> hunting success

In [ ]:
edges = [
    ("xp_level", "hunting_type"),
    ("hunting_type", "hunting_success")
]